In [1]:
%%sh
python convert.py

processing /home/jovyan/jsons/20_03_21__04_00_04.json
56112 records read from json
+-------+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|   shop|                name|weight| price|old_price|price_per_kg|old_price_per_kg|discount|cat|product_id| type|              datetm|                 url|
+-------+--------------------+------+------+---------+------------+----------------+--------+---+----------+-----+--------------------+--------------------+
|karusel| Антиперспирант-к...|    49|199.99|     null|     4081.43|            null|    null| 15|   1007746|piece|2020-03-21 05:48:...|/products/35828-a...|
|  metro| Антиперспирант-к...|    49| 111.9|    190.0|     2283.67|        3877.551|   41.11| 15|    137598|piece|2020-03-21 04:14:...|/products/35828-a...|
|karusel| Бальзам для особ...|   225| 97.99|     null|      435.51|            null|    null| 15|   1019125|piece|2020-03-21 05:48:.

20/03/21 12:38:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [3]:
from pyspark.sql.dataframe import DataFrame
DataFrame.old_show = getattr(DataFrame, 'old_show', DataFrame.show)
def show(self, n=20, truncate=20, vertical=False):
    self.persist()
    print(self.count())
    self.old_show(n, truncate=truncate, vertical=vertical)
    return self

DataFrame.show = show

import random
def sql(self, q):
    name = 'table_{}'.format(random.randint(0000, 9999))
    self.createOrReplaceTempView(name)
    q = q.format(T=name)
    print(q)
    return df.sql_ctx.sql(q)
DataFrame.sql = sql


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType, StringType, BooleanType
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()





In [5]:
df = spark.read.json('./jsons/20_03_30__18_31_18.json')

In [7]:
df.select('address', 'area', 'area_kitchen', 'area_live', 'categoryId', 'categorySlug', 'commission', 'commission_amount', 'datetm','deposit', 'dynx_category', 'dynx_pagetype', 'dynx_price', 'dynx_prodid', 'dynx_region', 'dynx_user', 'dynx_vertical',).show(100)

114
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|             address|   area|area_kitchen|area_live|categoryId|categorySlug| commission|commission_amount|              datetm| deposit|dynx_category|dynx_pagetype|dynx_price|dynx_prodid|         dynx_region|dynx_user|dynx_vertical|
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|Санкт-Петербург, ...|  35 м²|        6 м²|    25 м²|        24|    kvartiry|       null|             null|2020-03-30 18:31:...|    null|     kvartiry|         item|      1800| 1766646931|     sankt-peterburg|        b|            1|
|Санкт-Петербург, ...|  25 м²|        null|     null|       

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, dynx_user: string, dynx_vertical: bigint]

In [27]:
timeData='[ 24-янв-17 07:24 ]'    ### lower-case
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.utf8')
result = datetime.datetime.strptime(timeData, u'[ %d-%b-%y  %H:%M ]')
print(result)

Error: unsupported locale setting

In [21]:
def p(s):
    if 'сегодня' in s:
        return '1'
    return 'b'

In [29]:
p('сегодня в 20:41 ')

'1'

In [6]:
df.drop('dynx_user', 'dynx_vertical', 'vertical', 'withDelivery', 'dynx_category', 'dynx_pagetype', 'dynx_category', 'categorySlug', 'categoryId', 'dynx_price', 'pageType', 'isASDClient', 'dynx_prodid') \
    .drop('locationId', 'microCategoryId', 'dynx_region') \
    .withColumnRenamed('kol_vo_krovatey', 'beds') \
    .withColumnRenamed('kol_vo_spalnykh_mest', 'sleep_places') \
    .withColumnRenamed('floors_count', 'floors') \
    .withColumnRenamed('commission_amount', 'comm_amt') \
    .withColumnRenamed('commission', 'comm') \
    .where('offer_type="Сдам" and lease_period="На длительный срок"') \
    .drop('offer_type', 'lease_period') \
    .select('posted_datetm', 'url') \
    .withColumn('posted', F.udf(p, StringType())('posted_datetm')) \
    .show(1000, False)

NameError: name 'p' is not defined

In [92]:
for col in df.columns:
    df.groupby(col).count().sort('count', ascending=False).show(5)

54
+--------------------+-----+
|             address|count|
+--------------------+-----+
|Санкт-Петербург, ...|    3|
|Санкт-Петербург, ...|    2|
|Санкт-Петербург, ...|    2|
|Санкт-Петербург, ...|    2|
|Санкт-Петербург, ...|    2|
+--------------------+-----+
only showing top 5 rows

35
+-----+-----+
| area|count|
+-----+-----+
|36 м²|    6|
|32 м²|    4|
|40 м²|    4|
|70 м²|    3|
|60 м²|    3|
+-----+-----+
only showing top 5 rows

26
+------------+-----+
|area_kitchen|count|
+------------+-----+
|        null|   11|
|       15 м²|    5|
|        7 м²|    5|
|       13 м²|    4|
|       12 м²|    4|
+------------+-----+
only showing top 5 rows

33
+---------+-----+
|area_live|count|
+---------+-----+
|     null|   12|
|    20 м²|    6|
|    16 м²|    3|
|    17 м²|    3|
|    60 м²|    3|
+---------+-----+
only showing top 5 rows

1
+----------+-----+
|categoryId|count|
+----------+-----+
|        24|   61|
+----------+-----+

1
+------------+-----+
|categorySlug|count|
+-------

61
+--------------------+------+------------+---------+----------+------------+-----------+-----------------+--------------------+-------+-------------+-------------+----------+-----------+---------------+-----+------------+----------+-----------+------+----------+---------+---------------+--------------------+------------------+----------+---------------+----------+----------------+--------+----------------+------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+------------+
|             address|  area|area_kitchen|area_live|categoryId|categorySlug| commission|commission_amount|              datetm|deposit|dynx_category|dynx_pagetype|dynx_price|dynx_prodid|    dynx_region|floor|floors_count|house_type|isASDClient|isShop|    itemID|itemPrice|kol_vo_krovatey|kol_vo_spalnykh_mest|      lease_period|locationId|microCategoryId|offer_type|       old_prise|pageType|   posted_datetm| rooms|       seller

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, floor: string, floors_count: string, house_type: string, isASDClient: bigint, isShop: bigint, itemID: bigint, itemPrice: bigint, kol_vo_krovatey: string, kol_vo_spalnykh_mest: string, lease_period: string, locationId: bigint, microCategoryId: bigint, offer_type: string, old_prise: string, pageType: string, posted_datetm: string, rooms: string, seller_name: string, seller_url: string, sub_price: string, text: string, title: string, url: string, views: string, withDelivery: bigint]

In [8]:
df.select('address', 'area', 'area_kitchen', 'area_live', 'categoryId', 'categorySlug', 'commission', 'commission_amount', 'datetm','deposit', 'dynx_category', 'dynx_pagetype', 'dynx_price', 'dynx_prodid', 'dynx_region', 'dynx_user', 'dynx_vertical',).show(100)

114
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|             address|   area|area_kitchen|area_live|categoryId|categorySlug| commission|commission_amount|              datetm| deposit|dynx_category|dynx_pagetype|dynx_price|dynx_prodid|         dynx_region|dynx_user|dynx_vertical|
+--------------------+-------+------------+---------+----------+------------+-----------+-----------------+--------------------+--------+-------------+-------------+----------+-----------+--------------------+---------+-------------+
|Санкт-Петербург, ...|  35 м²|        6 м²|    25 м²|        24|    kvartiry|       null|             null|2020-03-30 18:31:...|    null|     kvartiry|         item|      1800| 1766646931|     sankt-peterburg|        b|            1|
|Санкт-Петербург, ...|  25 м²|        null|     null|       

DataFrame[address: string, area: string, area_kitchen: string, area_live: string, categoryId: bigint, categorySlug: string, commission: string, commission_amount: string, datetm: string, deposit: string, dynx_category: string, dynx_pagetype: string, dynx_price: bigint, dynx_prodid: bigint, dynx_region: string, dynx_user: string, dynx_vertical: bigint]

In [18]:
df.select('floor', 'floors_count', 'house_type', 'isASDClient', 'isShop', 'itemID', 'itemPrice', 'kol_vo_krovatey', 'kol_vo_spalnykh_mest','korpus', 'korpus_stroenie', 'lease_period', 'locationId', 'microCategoryId', 'nazvanie_novostroyki', 'nazvanie_obekta_nedvizhimosti').show(100)

58
+-----+------------+----------+-----------+------+----------+---------+---------------+--------------------+------+-----------------+------------------+----------+---------------+--------------------+-----------------------------+
|floor|floors_count|house_type|isASDClient|isShop|    itemID|itemPrice|kol_vo_krovatey|kol_vo_spalnykh_mest|korpus|  korpus_stroenie|      lease_period|locationId|microCategoryId|nazvanie_novostroyki|nazvanie_obekta_nedvizhimosti|
+-----+------------+----------+-----------+------+----------+---------+---------------+--------------------+------+-----------------+------------------+----------+---------------+--------------------+-----------------------------+
|    7|          16| Кирпичный|          0|     0|1844735206| 14470000|           null|                null|  null|             null|              null|    653240|           4920|                null|                         null|
|   16|          19|Монолитный|          0|     0|1901762309|    24000|  

DataFrame[floor: string, floors_count: string, house_type: string, isASDClient: bigint, isShop: bigint, itemID: bigint, itemPrice: bigint, kol_vo_krovatey: string, kol_vo_spalnykh_mest: string, korpus: bigint, korpus_stroenie: string, lease_period: string, locationId: bigint, microCategoryId: bigint, nazvanie_novostroyki: bigint, nazvanie_obekta_nedvizhimosti: string]

In [25]:
df.select('nomer_kvartiry', 'offer_type', 'ofitsialnyy_zastroyshchik', 'old_prise', 'otdelka', 'pageType', 'posted_datetm', 'proektnaya_deklaratsiya','rooms', 'seller_id', 'seller_name', 'seller_url', 'sub_price', 'text', 'tip_uchastiya', 'title', 'type', 'url', 'vertical', 'views', 'withDelivery').show(100)

58
+--------------+----------+-------------------------+----------------+--------+--------+----------------+-----------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------+--------------------+--------+-----------+------------+
|nomer_kvartiry|offer_type|ofitsialnyy_zastroyshchik|       old_prise| otdelka|pageType|   posted_datetm|proektnaya_deklaratsiya| rooms|           seller_id|         seller_name|          seller_url|           sub_price|                text|tip_uchastiya|               title|       type|                 url|vertical|      views|withDelivery|
+--------------+----------+-------------------------+----------------+--------+--------+----------------+-----------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-----------+--------------------+

DataFrame[nomer_kvartiry: string, offer_type: string, ofitsialnyy_zastroyshchik: string, old_prise: string, otdelka: string, pageType: string, posted_datetm: string, proektnaya_deklaratsiya: string, rooms: string, seller_id: string, seller_name: string, seller_url: string, sub_price: string, text: string, tip_uchastiya: bigint, title: string, type: string, url: string, vertical: string, views: string, withDelivery: bigint]

In [16]:
df.select('seller_id').show()

58
+--------------------+
|           seller_id|
+--------------------+
|                user|
|                user|
|                user|
|mkelite?gdlkerfdn...|
|                user|
|                user|
|                user|
|mirkvru?gdlkerfdn...|
|                user|
|                user|
|                user|
|                user|
|psk-info?gdlkerfd...|
|i178394876?gdlker...|
|cds-spb-ru?gdlker...|
|landrin-dom?gdlke...|
|psk-info?gdlkerfd...|
|                user|
|                user|
|                user|
+--------------------+
only showing top 20 rows



DataFrame[seller_id: string]

In [6]:
# total products per shop
df.groupBy("shop").pivot("date").agg(F.count('product_id')).show()


4
+-------+--------+--------+--------+--------+--------+
|   shop|20-03-16|20-03-17|20-03-18|20-03-19|20-03-21|
+-------+--------+--------+--------+--------+--------+
| prisma|    8991|    8689|    8629|    8660|    8680|
|  metro|   19599|   19645|   19365|   19702|   19727|
|karusel|   12501|   13139|   12450|   12642|   12803|
|  lenta|   14394|   14176|   13748|   14087|   14041|
+-------+--------+--------+--------+--------+--------+



DataFrame[shop: string, 20-03-16: bigint, 20-03-17: bigint, 20-03-18: bigint, 20-03-19: bigint, 20-03-21: bigint]

In [7]:
# total products with discounts per shop 
df.where('old_price is not null').groupBy("shop").pivot("date").agg(F.count('product_id')).show()


4
+-------+--------+--------+--------+--------+--------+
|   shop|20-03-16|20-03-17|20-03-18|20-03-19|20-03-21|
+-------+--------+--------+--------+--------+--------+
| prisma|     997|     934|     908|     783|     759|
|  metro|    2707|    2813|    2762|    2833|    2988|
|karusel|    1836|    2022|    1629|    1141|    1802|
|  lenta|    1513|    1531|    1490|    1430|    1305|
+-------+--------+--------+--------+--------+--------+



DataFrame[shop: string, 20-03-16: bigint, 20-03-17: bigint, 20-03-18: bigint, 20-03-19: bigint, 20-03-21: bigint]

In [8]:
# total average price per shop 
df.groupBy("shop").pivot("date").agg(F.sum('price')/F.count('product_id')).show()


4
+-------+------------------+------------------+------------------+------------------+------------------+
|   shop|          20-03-16|          20-03-17|          20-03-18|          20-03-19|          20-03-21|
+-------+------------------+------------------+------------------+------------------+------------------+
| prisma|188.10001347850155|188.34344018157324|189.03996892411402|192.07068617674275|192.57394608264696|
|  metro|  291.260432704589| 287.4751096981541| 289.6296090938372| 289.3170404041156|284.33816950983623|
|karusel|163.46044942153262|162.95179831070737|166.14586250631686| 168.7916300962232|166.81134088776088|
|  lenta|212.23314121221028|210.92874393766877|213.27697006009296|213.59038444901407| 213.3129290605269|
+-------+------------------+------------------+------------------+------------------+------------------+



DataFrame[shop: string, 20-03-16: double, 20-03-17: double, 20-03-18: double, 20-03-19: double, 20-03-21: double]